# Задания по уроку 5.2

In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              RandomTreesEmbedding, VotingClassifier)
from sklearn.linear_model import LogisticRegression

Все задания выполняются на основе датасета UCI ML Breast Cancer Wisconsin. (https://goo.gl/U2Uwz2)  
Все признаки являются числовыми.

In [2]:
X, y = load_breast_cancer(return_X_y=True)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

## Задачи с проверкой кода

1) Реализовать функцию нахождения самого частого элемента в массиве (голосование)

In [4]:
def get_most_frequent_value(sequence):
    unique, counts = np.unique(sequence, return_counts=True)
    d = dict(zip(unique, counts))
    i = list(d.values()).index(max(counts))

    return list(d.keys())[i]

In [5]:
get_most_frequent_value([1, 2, 3, 3, 3, 4, 4])

3

2) Реализовать функцию, выполняющую бутстреп.

In [6]:
def bootstrap(values, new_dataset_size):
    """
    Creates a new dataset from the old one using bootstrap.
    
    :arg values: np.array of shape (n_objects, n_features), input objects
    :arg new_dataset_size: int, number of elements in the resulting array
    
    :return: bootstraped_dataset: np.array of shape (new_dataset_size, n_features)
    """
    # TODO your code here
    Result_arr = np.empty((0,values.shape[1]), int)
    for i in range(new_dataset_size):
        Result_arr = np.append(Result_arr, values[np.random.choice(range(0,values.shape[0]),1)], axis = 0)
    return Result_arr

In [7]:
matr = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [8]:
bootstrap(matr, 5)

array([[4, 5, 6],
       [1, 2, 3],
       [4, 5, 6],
       [7, 8, 9],
       [1, 2, 3]])

3) Реализовать фукнцию для подсчета произвольной метрики из Scikit-Learn в режиме OOB. Подсказка: `oob_decision_function_`

In [10]:
# Write-up
def estimate_oob_metric(forest, metric):
    """
    Computes any classification metric in the out-of-bag mode.
        :arg forest: Scikit-Learn ensemble model
        :arg metric: callable with two arguments that returns a float
        
        :return oob_metric_value: float
    """
    return forest.oob_score_

In [11]:
forest = RandomForestClassifier(n_estimators=100, max_depth=8, criterion='entropy', oob_score=True, random_state=42)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_train)
f1 = f1_score(y_train, y_pred)

estimate_oob_metric(forest, f1)

0.967032967032967

## Задачи с проверкой ответа

В следующих заданих требуется так или иначе изменить параметры дерева решений и оценить результат.  
Как метрику качества будем использовать F1-score. В каждом задании необходимо поменять __только один__ параметр.  

Случайный лес — стохастический алгоритм, а значит при каждом запуске может выдавать различный результат. В дальнейших задачах __необходимо__ выставить случайность: добавить параметр `random_state=8` как аргумент при создании ансамблей.

In [12]:
def compute_metric(clf, X_train=X_train, y_train=y_train,
                   X_test=X_test, y_test=y_test):
    np.random.seed(42)
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred), 6)

Посчитаем значение метрики со стандартными параметрами.

In [13]:
forest = RandomForestClassifier(random_state=8)
compute_metric(forest)

0.965035

1) Напишите индекс самого важного признака по мнению случайного леса. Важности признаков указаны в переменной feature_importances_ случайного леса.

In [23]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
print('Важность признаков:', forest.feature_importances_)
max_value = max(forest.feature_importances_)
print('Индекс самого важного признака:', list(forest.feature_importances_).index(max_value))

Важность признаков: [0.04697058 0.01525873 0.05493599 0.0425965  0.0067247  0.01454576
 0.0713846  0.07780402 0.00351887 0.00447667 0.01115337 0.00470187
 0.01441008 0.02642848 0.00383245 0.00472714 0.00365893 0.00463324
 0.00478218 0.00542557 0.12163477 0.02637191 0.12802428 0.10325599
 0.01185224 0.01798744 0.03950645 0.10840577 0.015183   0.00580841]
Индекс самого важного признака: 22


2) Увеличьте количество в случайном лесе до 100. Как изменилось качество? Укажите его.

In [24]:
forest = RandomForestClassifier(random_state=8, n_estimators=100)
compute_metric(forest)

0.965035

3) Выставите количество деревьев в лесе равным 100 и включите режим подсчета out-of-bag score. Укажите полученное значение метрики accuracy, округленного до 6 знака после запятой.

In [27]:
def calc_accuracy(y_true, y_pred):
    P = np.sum(y_true == 1)
    N = np.sum(y_true == 0)
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    acc = (TP + TN) / (P + N)
    
    return acc

In [29]:
forest = RandomForestClassifier(random_state=8, n_estimators=100, oob_score=True)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_train)
acc = calc_accuracy(y_train, y_pred)
print('acc = {}%'.format(acc * 100))

acc = 100.0%


4) Выключите режим random subspaces в случайном лесе. Укажите качество.

In [30]:
forest = RandomForestClassifier(n_estimators=100, oob_score=True)
compute_metric(forest)

0.972222

5) Выключите режим бутстрапа и переобучите случайный лес. Укажите качество.

In [31]:
forest = RandomForestClassifier(n_estimators=100, bootstrap=False)
compute_metric(forest)

0.965035

6) Какая средняя глубина у деревьев, которые получаются в случайном лесе со стандартными параметрами при его обучении на этой обучающей выборке?

In [38]:
sum_depth = 0
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
for i in range(forest.n_estimators):
    sum_depth += forest.estimators_[0].tree_.max_depth
print('Средняя глубина деревьев леса =', sum_depth/forest.n_estimators)

Средняя глубина деревьев леса = 9.0
